Plant Disease Detection 
Access: open, available at New Plant Disease Dataset 

Data Description 

The data records contain 54,309 images. The images span 14 crop species: Apple, Blueberry, Cherry, Corn, Grape, Orange, Peach, Bell Pepper, Potato, Raspberry, Soybean, Squash, Strawberry, Tomato. In containes images of 17 fungal diseases, 4 bacterial diseases, 2 mold (oomycete) diseases, 2 viral disease, and 1 disease caused by a mite. 12 crop species also have images of healthy leaves that are not visibly affected by a disease  

Predictive task  

The predictive task can change depending on the images selected: one can predict plant disease vs plant good health or plant type.  

Related works:  

[1] Schuler, Joao Paulo Schwarz, et al. "Color-aware two-branch dcnn for efficient plant disease classification." MENDEL. Vol. 28. No. 1. 2022.  

[2] Mohanty, Sharada P., David P. Hughes, and Marcel Salathé. "Using deep learning for image-based plant disease detection." Frontiers in plant science 7 (2016): 1419.

In [1]:
!pip install opendatasets

import opendatasets as od
od.download("https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset/download?datasetVersionNumber=2")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: guglielmotedeschi
Your Kaggle Key: ··········


100%|██████████| 2.70G/2.70G [00:32<00:00, 87.9MB/s]



{"username":"`guglielmotedeschi`",


"key":"`5306be7ca91268007d1f0059dbdb0b68`"}

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os



In [2]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [4]:
train_dir = "/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_dir = "/content/new-plant-diseases-dataset/test"
diseases = os.listdir(train_dir)

In [ ]:
print("Total disease classes are: {}".format(len(diseases)))
print(diseases)

Total disease classes are: 38
['Blueberry___healthy', 'Raspberry___healthy', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Grape___healthy', 'Apple___Apple_scab', 'Orange___Haunglongbing_(Citrus_greening)', 'Potato___Early_blight', 'Cherry_(including_sour)___Powdery_mildew', 'Tomato___Early_blight', 'Potato___Late_blight', 'Corn_(maize)___Northern_Leaf_Blight', 'Apple___healthy', 'Peach___Bacterial_spot', 'Corn_(maize)___Common_rust_', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___Esca_(Black_Measles)', 'Strawberry___Leaf_scorch', 'Tomato___Target_Spot', 'Apple___Cedar_apple_rust', 'Pepper,_bell___Bacterial_spot', 'Tomato___Late_blight', 'Soybean___healthy', 'Strawberry___healthy', 'Peach___healthy', 'Tomato___Bacterial_spot', 'Pepper,_bell___healthy', 'Tomato___Leaf_Mold', 'Corn_(maize)___healthy', 'Apple___Black_rot', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Squash___Powdery_mildew', 'Tomato___Septoria_leaf_spot', 'Potato___healthy'

#A look on Data


In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1

In [ ]:
print(f"Unique Plants are: \n{plants}")


Unique Plants are: 
['Tomato', 'Strawberry', 'Apple', 'Corn_(maize)', 'Potato', 'Orange', 'Blueberry', 'Soybean', 'Cherry_(including_sour)', 'Pepper,_bell', 'Peach', 'Raspberry', 'Squash', 'Grape']


In [ ]:
print("Number of plants: {}".format(len(plants)))


Number of plants: 14


In [ ]:
print("Number of diseases: {}".format(NumberOfDiseases))


Number of diseases: 26


In [ ]:
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
    
# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
# plotting number of images available for each disease
index = [n for n in range(38)]
plt.figure(figsize=(20, 5))
plt.bar(index, [n for n in nums.values()], width=0.3)
plt.xlabel('Plants/Diseases', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
plt.xticks(index, diseases, fontsize=5, rotation=90)
plt.title('Images per each class of plant disease')

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

There are 70295 images for training


#to Tensor


In [5]:
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchvision import transforms

transform = transforms.Compose([transforms.Resize(256),
                                    #transforms.CenterCrop(224),
                                    #transforms.RandomHorizontalFlip(),
                                    #transforms.RandomRotation(10),
                                    transforms.ToTensor()])


train = ImageFolder(train_dir, transform=transform) # ToTensor images come as PIL format, we want to turn into Torch tensors
valid = ImageFolder(valid_dir, transform=transform) 
test = ImageFolder(test_dir, transform=transform)

class_names= train.classes

In [ ]:
img, label = train[0]
print(img.shape, label)
print("n classes:",len(class_names))

torch.Size([3, 256, 256]) 0
n classes: 38


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train), size=[1]).item()
    img, label = train[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))
    plt.title(class_names[label])
    plt.axis(False);

Prepare a DataLoader

#nepsi to gpu


In [6]:
# for moving data into GPU (if available)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available:
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [7]:
device = get_default_device()
device


device(type='cuda')

In [8]:
from torch.utils.data import DataLoader # DataLoader permette di splittare il dataset in batches 

batch_size = 32

train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
valid_dl = DataLoader(valid, batch_size=batch_size, shuffle=True)

train_dl = DeviceDataLoader(train_dl, device)
test_dl = DeviceDataLoader(test_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)
# Let's check out what we've created
print(f"Dataloaders: {train_dl, test_dl}") 
print(f"Length of train dataloader: {len(train_dl)} batches of {batch_size}")
print(f"Length of test dataloader: {len(test_dl)} batches of {batch_size}")


Dataloaders: (<__main__.DeviceDataLoader object at 0x7fb4aec761d0>, <__main__.DeviceDataLoader object at 0x7fb5933e6e60>)
Length of train dataloader: 2197 batches of 32
Length of test dataloader: 2 batches of 32


# Building the model


## CNN

In [9]:
# convolution block with BatchNormalization
def NepsiBlock(in_channels, out_channels, pool=False):
    nepsi_layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        nepsi_layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*nepsi_layers)

In [24]:
class Plant(nn.Module):
    def __init__(self, in_channels: int, diseases: int):
        super().__init__()

        self.conv1 = NepsiBlock(in_channels, 64)
        self.conv2 = NepsiBlock(64, 128, pool=True)
        self.res1 = nn.Sequential(NepsiBlock(128,128),NepsiBlock(128,128))

        self.conv3 = NepsiBlock(128,256, pool=True)
        self.conv4 = NepsiBlock(256, 512, pool=True)
        self.res2 = nn.Sequential(NepsiBlock(512,512), NepsiBlock(512,512))

        self.classifier = nn.Sequential((nn.MaxPool2d(4)),
                                        nn.Flatten(),
                                        nn.Linear(512, diseases))
        

    def forward(self,x):
      out = self.conv1(x)
      out = self.conv2(out)
      out = self.res1(out) + out
      out = self.conv3(out)
      out = self.conv4(out)
      out = self.res2(out) + out
      out = self.classifier(out)
      return out

torch.manual_seed(42)
#model = to_device(convo_nepsi(3, len(train.classes)), device) 

convo_nepsi = to_device (Plant(in_channels=3,
                               diseases=len(class_names)), device)
  
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=convo_nepsi.parameters(), 
                             lr=0.01)

convo_nepsi

Plant(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True

Function for Train

In [48]:
def train_step(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer, accuracy_fn):
    
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(data_loader):
        # 1. Forward pass
        y_pred = model(X)
        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels
        # 3. Optimizer zero grad
        optimizer.zero_grad()
        # 4. Loss backward
        loss.backward()
        # 5. Optimizer step
        optimizer.step()  
        
        if batch%100 == 0:
          print(batch)
          print(f'train loss {train_loss / (batch+1)}')
          print(f'train acc:{train_acc/ (batch+1)}')
          #print(y)
          #print(y_pred.argmax(dim=1))
          
    
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")



    # Calculate loss and accuracy per epoch and print out what's happening
      
def test_step(data_loader: torch.utils.data.DataLoader, model: torch.nn.Module, loss_fn: torch.nn.Module, accuracy_fn):
    model.eval() # put model in eval mode
    test_loss, test_acc = 0, 0
   
    # Turn on inference context manager
    with torch.no_grad(): 
        for X, y in data_loader:
            # 1. Forward pass
            test_pred = model(X)
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
            print(f'test loss {test_loss / (batch+1)}')
            print(f'test acc:{test_acc/ (batch+1)}')
          
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [52]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time: {total_time:.3f} seconds")
    return total_time

def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.
    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.
    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred))
    return acc

In [53]:
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Make predictions with the model
            y_pred = model(X)
            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)
        
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [55]:
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
train_time_start = timer()

epochs = 3
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dl, 
        model=convo_nepsi, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_dl,
        model=convo_nepsi,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

train_time_end= timer()
total_train_time_model_1 = print_train_time(start=train_time_start, end=train_time_end)

Epoch: 0
---------
0
train loss 0.00012485039769671857
train acc:1.0
100
train loss 0.03295497166101018
train acc:0.9925742574257426
200
train loss 0.024546349054097305
train acc:0.9936256218905473
300
train loss 0.024479153044570302
train acc:0.9937707641196013
400
train loss 0.03292384998404004
train acc:0.9917394014962594
500
train loss 0.043447784144370914
train acc:0.9893962075848304
600
train loss 0.04910696868939998
train acc:0.9882487520798668
700
train loss 0.05100515313469908
train acc:0.9876069900142653
800
train loss 0.05394164078565603
train acc:0.9868133583021224
900
train loss 0.058543639144601894
train acc:0.9858837402885683
1000
train loss 0.058902142639584563
train acc:0.9855769230769231
1100
train loss 0.05832568935482776
train acc:0.9857799727520435
1200
train loss 0.05657707429223418
train acc:0.9861053288925895
1300
train loss 0.05828145386459941
train acc:0.9858762490392006
1400
train loss 0.05941415972363526
train acc:0.9855906495360457
1500
train loss 0.0598206